# Strange regression mephi-2

https://www.kaggle.com/c/strange-regression-mephi-2

Импортируем джентельменский набор

In [19]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

Загружаем данные и начинаем работу с ними

In [2]:
data_train = pd.read_csv("data/kagle_train.csv", parse_dates=['start_time'], index_col='item_id')

In [3]:
data_test = pd.read_csv('data/regr_test.csv', parse_dates=['start_time'])

In [4]:
data_train.sample(5)

,id,start_time,title,price,owner_type,category,subcategory,param1,param2,param3,region,item_views
item_id,,,,,,,,,,,,
5705898845763758734,329407,2016-12-26 17:37:44,Эрго-рюкзак love carry,500,Company,Личные вещи,Товары для детей и игрушки,Игрушки,NaN,NaN,Ивановская область,3.258097
2724359157403861389,91688,2016-12-28 18:16:00,30 от цены. Электронная книга,3800,Private,Бытовая электроника,Планшеты и электронные книги,Электронные книги,NaN,NaN,Самарская область,4.584967
3177615282527920911,350468,2016-12-28 18:10:17,Сапоги замшевые carlo pazolini 39р,5000,Private,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,39,Красноярский край,3.688879
294337942887387431,94285,2016-12-26 08:02:40,Черепаха в дар,0,Company,Животные,Аквариум,NaN,NaN,NaN,Курская область,3.891820
152786503366182060,344389,2016-12-28 21:38:00,Шикарное платье,1500,Private,Личные вещи,Детская одежда и обувь,Для девочек,Платья и юбки,110-116 см (4-6 лет),Нижегородская область,4.158883


In [5]:
data_test.sample(5)

,id,start_time,title,price,item_id,owner_type,category,subcategory,param1,param2,param3,region
12575,157424,2016-12-26 15:37:06,Мзса-К 233,47402,310997009152403822,Shop,Транспорт,Запчасти и аксессуары,Прицепы,NaN,NaN,Санкт-Петербург
40251,350700,2016-12-28 16:52:34,"Nissan Almera, 2004",265000,2150371097303435466,Company,Транспорт,Автомобили,С пробегом,Nissan,Almera,Краснодарский край
39189,343276,2016-12-27 10:25:38,Магнитола Sony CFD-RG880CP,5000,4183488402914497302,Company,Бытовая электроника,Аудио и видео,"Музыкальные центры, магнитолы",NaN,NaN,Краснодарский край
16893,224337,2016-12-27 15:51:33,Айфон 5 s,13000,5907889794881664344,Private,Бытовая электроника,Телефоны,iPhone,NaN,NaN,Краснодарский край
33120,283950,2016-12-28 16:41:03,Дом 54 м? на участке 6 сот.,3750000,6496950207402085953,Shop,Недвижимость,"Дома, дачи, коттеджи",Продам,Дом,NaN,Калужская область


## Выделяем признаки:
- количество заглавных букв
- количество "!"
- плотность заглавных букв и '!'
- количество латинских символов
- длина заголовка
- количество цифр
- для текста:
    - избавляемся от пунктуации
    - нормализуем слова
    - Генерируем CountVectorized
- Время:
    - вводим гармонические функции
    - выделяем день недели

In [6]:
def count_uppercase(text):
    counter = 0
    for c in text:
        if c.isupper():
            counter += 1
            
    return counter

In [7]:
def count_exclaim_mark(text):
    counter = 0
    for c in text:
        if c == '!':
            counter += 1
            
    return counter

In [8]:
import re

english_check = re.compile(r'[a-z]')

def count_eng(text):
    counter = 0
    for c in text:
        if english_check.match(c):
            counter += 1
            
    return counter

In [9]:
def count_digits(text):
    counter = 0
    for c in text:
        if c.isdigit():
            counter += 1
            
    return counter

In [10]:
def make_harmonic_features(value, period=24):
    value = (value / period) * 2 * np.pi
    return np.cos(value), np.sin(value)

In [11]:
import pymorphy2 as morphy
import string
from functools import lru_cache

morpher = morphy.MorphAnalyzer()

In [12]:
@lru_cache(maxsize=100000)
def get_normal_form (word):
    return morpher.normal_forms(word)[0]

In [13]:
def text_normalizer(text):
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower()
    words = text.split()
    normalized_text = ''
    for word in words:
        normalized_text += get_normal_form(word) + ' '
        
    return normalized_text.rstrip()

In [14]:
title_features_columns = ['title_uc_!', 'title_uc_!_density', 'title_eng_count', 'title_len', 'title_digits', 'price']

In [15]:
def feature_engineering(df):

    df['title_uc'] = df['title'].apply(count_uppercase)
    df['title_!'] = df['title'].apply(count_exclaim_mark)
    df['title_len'] = df['title'].apply(len)
    df['title_uc_!_density'] = (
        df['title_!'] + df['title_uc']) / df['title_len']
    df['title_eng_count'] = df['title'].apply(count_eng)
    df['title_digits'] = df['title'].apply(count_digits)

    df['title'] = df['title'].apply(text_normalizer)

    df['cos_time'] = df['start_time'].apply(
        lambda x: make_harmonic_features(x.hour + x.minute / 60)[0])
    df['sin_time'] = df['start_time'].apply(
        lambda x: make_harmonic_features(x.hour + x.minute / 60)[1])
    df['week_day'] = df['start_time'].apply(lambda x: x.isoweekday())

In [16]:
from multiprocessing import Pool

In [17]:
%%notify
%%time

feature_engineering(data_train)
feature_engineering(data_test)

CPU times: user 43.2 s, sys: 539 ms, total: 43.7 s
Wall time: 43.2 s


<IPython.core.display.Javascript object>

## Все категориальные признаки => OneHot
предварительно избавившись от NaN

In [32]:
data_train.fillna('nan', inplace=True)
data_test.fillna('nan', inplace=True)

In [23]:
cat_features_cols = ['owner_type', 'category', 'subcategory', 'param1', 'param2', 'param3', 'region', 'week_day']

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [31]:
oh_encoder = OneHotEncoder(handle_unknown='ignore')

cat_features_train = oh_encoder.fit_transform(data_train.loc[:, cat_features_cols])
cat_features_test = oh_encoder.transform(data_test.loc[:, cat_features_cols])

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

## Текст => СountVectorize

In [34]:
text_vectorizer = CountVectorizer()

title_features_train = text_vectorizer.fit_transform(data_train['title'])
title_features_test = text_vectorizer.transform(data_test['title'])

#### Собираем все признаки

In [36]:
ready_feature_cols = ['price', 'title_uc', 'title_!', 'title_len',
                      'title_uc_!_density', 'title_eng_count', 'title_digits', 'cos_time', 'sin_time']

In [37]:
features_train = sp.hstack((title_features_train, cat_features_train, data_train.loc[:, ready_feature_cols]))
features_test = sp.hstack((title_features_test, cat_features_test, data_test.loc[:, ready_feature_cols]))

## Используемая модель - RandomForest с бустингом от xgboost

In [38]:
import xgboost as xgb

In [40]:
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    features_train, data_train['item_views'], test_size=0.20, random_state=648, shuffle=True)

In [85]:
xgb_params = {
#     'objective': 'reg:squarederror',
    'max_depth': 6,
    'eta': 0.05,
    'booster': 'gbtree',
    
    'alpha': 1.0,
    'lambda': 0.1,
    
    'subsample': 0.9,
    'colsample_bytree': 0.5,
    'colsample_bylevel': 0.3,
    
    'seed': 648,
    'nthread': 4,
    'eval_metric':'rmse'
}

num_rounds = 10000

In [86]:
%%notify
%%time

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

watchlist = [(dtrain, 'train'), (dtest, 'eval')]
eval_res ={}
gbdt = xgb.train(xgb_params, dtrain,
                 num_rounds, watchlist,
                 early_stopping_rounds=5,
                 verbose_eval=200,
                 evals_result=eval_res)

[0]	train-rmse:3.00464	eval-rmse:3.00913
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 5 rounds.
[200]	train-rmse:0.638885	eval-rmse:0.6415
[400]	train-rmse:0.616263	eval-rmse:0.621961
[600]	train-rmse:0.602757	eval-rmse:0.611349
[800]	train-rmse:0.59263	eval-rmse:0.603855
[1000]	train-rmse:0.584938	eval-rmse:0.598599
[1200]	train-rmse:0.578253	eval-rmse:0.594164
[1400]	train-rmse:0.573133	eval-rmse:0.591028
[1600]	train-rmse:0.568161	eval-rmse:0.587976
[1800]	train-rmse:0.564106	eval-rmse:0.585634
[2000]	train-rmse:0.560217	eval-rmse:0.583513
[2200]	train-rmse:0.556559	eval-rmse:0.581477
[2400]	train-rmse:0.553375	eval-rmse:0.579813
[2600]	train-rmse:0.550078	eval-rmse:0.578114
[2800]	train-rmse:0.547381	eval-rmse:0.576797
[3000]	train-rmse:0.544579	eval-rmse:0.575494
[3200]	train-rmse:0.542125	eval-rmse:0.574375
[3400]	train-rmse:0.539811	eval-rmse:0.57335
[3600]	train-rmse:0.537477	eval-rmse:0.5723

<IPython.core.display.Javascript object>

# Сабмит

In [87]:
dtest_final = xgb.DMatrix(features_test)

y_pred = gbdt.predict(dtest_final)

pd.DataFrame({'id': data_test.id, 'item_views':y_pred}).to_csv('xgb_prediction.csv', index=None)